# Spotify EDA

Spotify is a music pltaform that allows you to stream your favorite songs in praticaly any device.
Spotify also allows you to request your personal data to them, that will include your paymment info as well all the usage information of the user. 

Once you request the data you'll just have to wait around 30 days and they'll email it back to you, simple as that.

The data comes in JSON format and is divided in personal info and the info about the songs and playlists that you created and so on. 

In [16]:
# Let's import some packages
import pandas as pd
import numpy as np
import plotly.express as px
import calplot
import mitosheet
from datetime import datetime

I'll use the data from the straming taht contains the amount the time each song was played as well the data about the tracks, it will be userfull to get the data about the songs later.

In [19]:
streaming = pd.read_json('MyData/StreamingHistory0.json')
tracks = pd.read_json('MyData/tracks.json')
streaming.head()

,endTime,artistName,trackName,msPlayed
0,2020-12-30 00:14,Thomas Sanders,The Things We Used to Share,138645
1,2021-03-11 02:24,Gentle Whispering,Sleep Doctor Exam Asmr,0
2,2021-03-11 02:55,Gentle Whispering,Sleep Doctor Exam Asmr,1856528
3,2021-03-15 20:06,amazarashi,月曜日,2412
4,2021-03-15 20:07,the peggies,Bokurano Senso,2991


In [20]:
tracks.head()

,artist,album,track,uri
0,Cidergirl,SODA POP FANCLUB 1,エバーグリーン,spotify:track:3azJLt5FsOam6070rcuxJa
1,THE ORAL CIGARETTES,FIXION,Everything,spotify:track:3uaLzkODN7xLUkcQDSg15P
2,frederic,Frederhythm2,Shinsense,spotify:track:1GOPolIMSioKeNOwrNFDJg
3,Dropkick Murphys,The Warrior's Code,Your Spirit's Alive,spotify:track:56IfZE4yzzv9GcqPoooI4k
4,EDOGA-SULLIVAN,WONDERFUL WONDER,WONDERFUL WONDER,spotify:track:7t3kgQ9mF3t4pyuWxm02pX


In [23]:
streaming["UniqueID"] = streaming["artistName"] + ":" + streaming["trackName"]
tracks["UniqueID"] = tracks["artist"] + ":" + tracks["track"]
songs = pd.merge(streaming, tracks[["album", "UniqueID"]], how = "left", on = ["UniqueID"])
songs.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album
0,2020-12-30 00:14,Thomas Sanders,The Things We Used to Share,138645,Thomas Sanders:The Things We Used to Share,NaN
1,2021-03-11 02:24,Gentle Whispering,Sleep Doctor Exam Asmr,0,Gentle Whispering:Sleep Doctor Exam Asmr,NaN
2,2021-03-11 02:55,Gentle Whispering,Sleep Doctor Exam Asmr,1856528,Gentle Whispering:Sleep Doctor Exam Asmr,NaN
3,2021-03-15 20:06,amazarashi,月曜日,2412,amazarashi:月曜日,NaN
4,2021-03-15 20:07,the peggies,Bokurano Senso,2991,the peggies:Bokurano Senso,NaN


In [24]:
# Converting the time Played to hours
songs["HoursPlayed"] = (songs["msPlayed"]/(1000*60*60))%24
songs.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album,HoursPlayed
0,2020-12-30 00:14,Thomas Sanders,The Things We Used to Share,138645,Thomas Sanders:The Things We Used to Share,NaN,0.038512
1,2021-03-11 02:24,Gentle Whispering,Sleep Doctor Exam Asmr,0,Gentle Whispering:Sleep Doctor Exam Asmr,NaN,0.000000
2,2021-03-11 02:55,Gentle Whispering,Sleep Doctor Exam Asmr,1856528,Gentle Whispering:Sleep Doctor Exam Asmr,NaN,0.515702
3,2021-03-15 20:06,amazarashi,月曜日,2412,amazarashi:月曜日,NaN,0.000670
4,2021-03-15 20:07,the peggies,Bokurano Senso,2991,the peggies:Bokurano Senso,NaN,0.000831


In [25]:
grouped_artists = songs.groupby("artistName")
artists = songs["artistName"].unique()
dummy_list =  []
for artist in artists:
    if grouped_artists.get_group(artist)["HoursPlayed"].sum() > 1:
        dummy_list.append(grouped_artists.get_group(artist))
    
most_played_songs = pd.concat(dummy_list, ignore_index=True)      

In [26]:
mitosheet.sheet(most_played_songs, analysis_to_replay="id-fapqqmgisw")

MitoWidget(analysis_data_json='{"analysisName": "id-wzzhpvohnb", "analysisToReplay": {"analysisName": "id-fapq…

In [14]:
import spotipy.util as util
import spotipy
import requests

from  configparser import ConfigParser
from spotipy.oauth2 import SpotifyClientCredentials

In [18]:
config = ConfigParser()
scope = 'user-read-recently-played'
config.read('config.cfg')
client_id = config.get('SPOTIFY', 'CLIENT_ID')
client_secret = config.get('SPOTIFY', 'CLIENT_SECRET')
user_id = config.get('SPOTIFY', 'USER_ID')
redirect_uri = 'http://localhost:5000/callback'

token = util.prompt_for_user_token(scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [16]:
track = "asd"
artist = "plo"
print(f'{track} {artist}')

asd plo


In [ ]:
def get_id(track, artist, token):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}',
    }
    track_artist = f'{track} {artist}'
    params = [
        ('q', track_artist ),
        ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 10)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

# Get track_id for streaming history    
merge["track_id"] = merge.apply(lambda x: get_id(x["trackName"], x["artistName"], token),axis=1)

In [11]:
df_merged_copy

,endTime,artistName,trackName,msPlayed,UniqueID,album,HoursPlayed,track_id
0,2021-03-11 02:24,Gentle Whispering,Sleep Doctor Exam Asmr,0,Gentle Whispering:Sleep Doctor Exam Asmr,NaN,0.000000,56PnFazA6pZwWJNxKpyZ9h
1,2021-03-11 02:55,Gentle Whispering,Sleep Doctor Exam Asmr,1856528,Gentle Whispering:Sleep Doctor Exam Asmr,NaN,0.515702,56PnFazA6pZwWJNxKpyZ9h
2,2021-03-31 14:25,Gentle Whispering,Asmr Exam in a Tranquil Garden,2275773,Gentle Whispering:Asmr Exam in a Tranquil Garden,NaN,0.632159,1YextYICihCehDyRviJAd2
3,2021-03-31 17:11,Gentle Whispering,Sleep Inducing Hair and Scalp Treatment Asmr,1525854,Gentle Whispering:Sleep Inducing Hair and Scalp Treatment Asmr,NaN,0.423848,6gBLeMYU4dBKrwfjmZ03V3
4,2021-04-28 19:43,Gentle Whispering,Enchantress Asmr,331116,Gentle Whispering:Enchantress Asmr,NaN,0.091977,25oZFmvXQZQiTnBNM47AER
...,...,...,...,...,...,...,...,...
2231,2021-12-26 16:58,Men At Work,Snakes And Ladders,196560,Men At Work:Snakes And Ladders,NaN,0.054600,7IoLEv4i0PeZHzlhRsThTR
2232,2021-12-26 17:22,Men At Work,It's A Mistake,286973,Men At Work:It's A Mistake,NaN,0.079715,1ArdfXwdvwzapms9p1wxEb
2233,2021-12-26 17:43,Men At Work,Down Under,220867,Men At Work:Down Under,NaN,0.061352,3ZZq9396zv8pcn5GYVhxUi
2234,2021-12-26 18:06,Men At Work,Man with Two Hearts,4214,Men At Work:Man with Two Hearts,NaN,0.001171,1nAHeNRcYlFqVicmrsfBlN


In [16]:
import mitosheet
mitosheet.sheet(df_merged_copy, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-68ec5968-222a-49f2-9eb6-9314bbe832f6", "code": {"imports…

In [16]:
mitosheet.sheet(df_merged_copy, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-84a19022-821a-4f02-8f51-5c633fbe92f0", "code": {"imports…

In [12]:
tracks_ids=df_merged_copy['track_id'].dropna()
tracks_ids

0       56PnFazA6pZwWJNxKpyZ9h
1       56PnFazA6pZwWJNxKpyZ9h
2       1YextYICihCehDyRviJAd2
3       6gBLeMYU4dBKrwfjmZ03V3
4       25oZFmvXQZQiTnBNM47AER
                 ...          
2231    7IoLEv4i0PeZHzlhRsThTR
2232    1ArdfXwdvwzapms9p1wxEb
2233    3ZZq9396zv8pcn5GYVhxUi
2234    1nAHeNRcYlFqVicmrsfBlN
2235    1LxrKXZ93NST4bWJ3i4LHJ
Name: track_id, Length: 2212, dtype: object

In [18]:
tracks_ids=list(tracks_ids)

In [15]:
sp = spotipy.Spotify(token)
features=sp.audio_features("56PnFazA6pZwWJNxKpyZ9h")
features[0]

{'danceability': 0.32,
 'energy': 0.0533,
 'key': 10,
 'loudness': -39.588,
 'mode': 0,
 'speechiness': 0.288,
 'acousticness': 0.00492,
 'instrumentalness': 0.00148,
 'liveness': 0.223,
 'valence': 0.138,
 'tempo': 64.849,
 'type': 'audio_features',
 'id': '56PnFazA6pZwWJNxKpyZ9h',
 'uri': 'spotify:track:56PnFazA6pZwWJNxKpyZ9h',
 'track_href': 'https://api.spotify.com/v1/tracks/56PnFazA6pZwWJNxKpyZ9h',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/56PnFazA6pZwWJNxKpyZ9h',
 'duration_ms': 2183231,
 'time_signature': 4}

In [17]:
features[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [24]:
columns = ['id', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
           'instrumentalness', 'liveness', 'valence', 'tempo', 'key', 'mode']

features_df = []
for track in tracks_ids:
    feature=sp.audio_features(track)[0]
    
    if feature is not None:
        features_df.append(feature)

In [30]:
features_dataframe=pd.DataFrame(features_df)
features_dataframe.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.320,0.0533,10,-39.588,0,0.2880,0.00492,0.001480,0.223,0.1380,64.849,audio_features,56PnFazA6pZwWJNxKpyZ9h,spotify:track:56PnFazA6pZwWJNxKpyZ9h,https://api.spotify.com/v1/tracks/56PnFazA6pZwWJNxKpyZ9h,https://api.spotify.com/v1/audio-analysis/56PnFazA6pZwWJNxKpyZ9h,2183231,4
1,0.320,0.0533,10,-39.588,0,0.2880,0.00492,0.001480,0.223,0.1380,64.849,audio_features,56PnFazA6pZwWJNxKpyZ9h,spotify:track:56PnFazA6pZwWJNxKpyZ9h,https://api.spotify.com/v1/tracks/56PnFazA6pZwWJNxKpyZ9h,https://api.spotify.com/v1/audio-analysis/56PnFazA6pZwWJNxKpyZ9h,2183231,4
2,0.339,0.0414,8,-41.819,1,0.3870,0.09320,0.000283,0.124,0.1940,64.208,audio_features,1YextYICihCehDyRviJAd2,spotify:track:1YextYICihCehDyRviJAd2,https://api.spotify.com/v1/tracks/1YextYICihCehDyRviJAd2,https://api.spotify.com/v1/audio-analysis/1YextYICihCehDyRviJAd2,2275774,4
3,0.354,0.0237,1,-44.809,1,0.1310,0.01100,0.003110,0.286,0.1260,104.156,audio_features,6gBLeMYU4dBKrwfjmZ03V3,spotify:track:6gBLeMYU4dBKrwfjmZ03V3,https://api.spotify.com/v1/tracks/6gBLeMYU4dBKrwfjmZ03V3,https://api.spotify.com/v1/audio-analysis/6gBLeMYU4dBKrwfjmZ03V3,3549680,4
4,0.407,0.1400,10,-39.902,0,0.0466,0.23200,0.001230,0.166,0.0754,111.235,audio_features,25oZFmvXQZQiTnBNM47AER,spotify:track:25oZFmvXQZQiTnBNM47AER,https://api.spotify.com/v1/tracks/25oZFmvXQZQiTnBNM47AER,https://api.spotify.com/v1/audio-analysis/25oZFmvXQZQiTnBNM47AER,1969000,3


Ref about all thi data [here](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)